In [1]:
#import necessary libraries
import numpy as np
import pandas as pd
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
color = sns.color_palette()
sns.set_style('darkgrid')
import warnings
from scipy import stats
from scipy.stats import norm, skew #for some statistics

ModuleNotFoundError: No module named 'sklearn'

In [ ]:
#import data
path = 'F:/Dropbox/Columbia MSBA/IEOR E4501 Tools for Analytics/Project/airbnb-price-prediction/train.csv'
data = pd.read_csv(path)

In [ ]:
data.head()

In [ ]:
print (len(data))

In [ ]:
data.columns

In [ ]:
len(data.columns)

In [ ]:
# check the dtypes of different types of variable.
pd.options.display.max_rows = 65
dtypes = data.dtypes.reset_index()
dtypes.columns = ["val_name", "Type"]
dtypes

In [ ]:
# divide the variables into numerical variables and categorical variables
num_var=['accommodates','bathrooms','cancellation_fee','first_view','host_response_rate','host_since','latitude','longitude','number_of_reviews','review_scores_rating',
        'zipcode','bedrooms','beds']
cate_var=['property_type','room_type','bed_type','cancellation_policy','city','host_has_profile_pic','host_identity_verified','instant_bookable','neighbourhood']

In [ ]:
# describe log_price# descr 
data['log_price'].describe()

In [ ]:
#calculete Target variable
data['Saleprice']=np.exp(data['log_price'])

In [ ]:
def drawHist(heights):
    plt.hist(heights, 20)
    plt.xlabel('Saleprice')
    plt.ylabel('Frequency')
    plt.title('Histogram of Saleprice')
    plt.show()
drawHist(data['Saleprice'])

In [ ]:
drawHist(data['log_price'])

In [ ]:
# plot probability distribution# plot  
fig = plt.figure()
res = stats.probplot(data['log_price'], plot=plt)
plt.show()

In [ ]:
# deal with dates
import time
import datetime
def dif_time(date):
    now_time = datetime.datetime.now()
    now_time = now_time.strftime('%Y-%m-%d')
    d2 = datetime.datetime.strptime(now_time, '%Y-%m-%d')
    if str(date) !='nan':
        d1 = datetime.datetime.strptime(str(date),'%Y-%m-%d')
        day = (d2 - d1).days
        return day
    else:
        return None

In [ ]:
#transform date to days
dates_to_transform=['first_review','host_since','last_review']
for col in dates_to_transform:
    data[col]=data[col].apply(dif_time)

In [ ]:
#calculate missing data
total_missing = data.isnull().sum()
missing_ratio = (data.isnull().sum()/data.isnull().count())
missing_data = pd.concat([total_missing, missing_ratio], axis=1, keys=['total_missing', 'missing_ratio'])
missing_data = missing_data.ix[missing_data['missing_ratio']>0]
missing_data = missing_data.sort_values(by='missing_ratio',ascending=False)
missing_data.head()

In [ ]:
#plot ratio of missing#plot ra 
f, ax = plt.subplots(figsize=(18, 10))
plt.xticks(rotation='90')
sns.barplot(x=missing_data.index, y=missing_data.missing_ratio)
plt.xlabel('properties', fontsize=18)
plt.ylabel('Ratio of missing values', fontsize=18)
plt.title('Ratio of missing data by properties', fontsize=20)

In [ ]:
#plot correlation matrix
import seaborn as sns
color = sns.color_palette()
sns.set_style('darkgrid')

def correlation_map(df):
    correlation = data.corr()
    plt.subplots(figsize=(15,10))
    plt.title('Correlation Map', fontsize=20)
    sns.heatmap(correlation, xticklabels=correlation.columns, yticklabels=correlation.columns)
    
correlation_map(data)

In [ ]:
#missing_values
# fill NA by 0
fillby0=['first_review','host_since','last_review']
for col in fillby0:
    data[col] = data[col].fillna(0)

In [ ]:
#fill NA by None
fillbynone=['neighbourhood']
for col in fillbynone:
    data[col] = data[col].fillna('None')

In [ ]:
#fill NA by numerical value
fillbymedian=['host_response_rate','review_scores_rating','number_of_reviews']
for col in fillbymedian:
    data[col] = data[col].fillna(data[col].transform(lambda x: x.fillna(x.median())))

In [ ]:
fillbymode=['host_has_profile_pic','bedrooms','beds','zipcode','instant_bookable']
for col in fillbymode:
    data[col] = data[col].fillna(data[col].mode()[0])

In [ ]:
#check if there is any missing_value left
def left_missing(dataset):
    val = dataset.isnull().sum().max()
    if val>0:
        raise ValueError('There is still missing_value left!')
left_missing(data)

In [ ]:
from sklearn.model_selection import train_test_split

train_data, test_data = train_test_split(data, test_size=0.2)

In [ ]:
train_data.head()


In [ ]:
test_data.head()

In [ ]:
print (len(train_data))
print (len(test_data))

In [ ]:
#if one column only has one value, consider to delete the column
train_columns_to_remove = [col for col in train_data.columns if train_data[col].nunique() == 1]
test_columns_to_remove = [col for col in test_data.columns if test_data[col].nunique() == 1]

In [ ]:
print(train_columns_to_remove)
print(test_columns_to_remove)

In [ ]:
for col in data.columns:
    if data[col].isnull().sum() > 0:
        rate = data[col].isnull().sum() * 100 / data.shape[0]
        print(f'Column {col} has {rate:.4f}% missing values.')
    if data[col].dtype == 'object':
        if (data[col] == 'not available in demo dataset').sum() > 0:
            rate = (data[col] == 'not available in demo dataset').sum() * 100 / data.shape[0]
            print(f'Column {col} has {rate:.4f}% values not available in dataset.')

In [ ]:
# deal with dates
import time
import datetime
def dif_time(date):
    now_time = datetime.datetime.now()
    now_time = now_time.strftime('%Y-%m-%d')
    d2 = datetime.datetime.strptime(now_time, '%Y-%m-%d')
    if str(date) != 'nan':
        d1 = datetime.datetime.strptime(str(date), '%Y-%m-%d')
        day = (d2 - d1).days
        return day
    else:
        return None

In [ ]:
data['last_review'] = data['last_review'].apply(dif_time)

In [ ]:
data['neighbourhood'].nunique()

In [ ]:
from sklearn.preprocessing import LabelEncoder

In [ ]:
for col in train_data.columns:
    print(col)
    lbl = LabelEncoder()
    lbl.fit(list(train_data[col].values.astype('str')) + list(test_data[col].values.astype('str')))
    train_data[col] = lbl.transform(list(train_data[col].values.astype('str')))
    test_data[col] = lbl.transform(list(test_data[col].values.astype('str')))

In [ ]:
from sklearn.model_selection import TimeSeriesSplit

In [ ]:
from sklearn.model_selection import KFold #Provides train/test indices to split data in train/test sets. Split dataset into k consecutive folds (without shuffling by default).
import lightgbm as lgb

In [ ]:
params = {"objective" : "regression", "metric" : "rmse", "max_depth": 8, "min_child_samples": 20, "reg_alpha": 0.2, "reg_lambda": 0.2,
        "num_leaves" : 257, "learning_rate" : 0.01, "subsample" : 0.9, "colsample_bytree" : 0.9, "subsample_freq ": 5}
n_fold = 10
folds = KFold(n_splits=n_fold, shuffle=False, random_state=42)
# Cleaning and defining parameters for LGBM
model = lgb.LGBMRegressor(**params, n_estimators = 20000, nthread = 4, n_jobs = -1)

In [ ]:
prediction = np.zeros(test_data.shape[0])

for fold_n, (train_index, test_index) in enumerate(folds.split(X)):
    print('Fold:', fold_n)
    #print(f'Train samples: {len(train_index)}. Valid samples: {len(test_index)}')
    X_train, X_valid = X.iloc[train_index], X.iloc[test_index]
    y_train, y_valid = y.iloc[train_index], y.iloc[test_index]
    

    model.fit(X_train, y_train, 
            eval_set=[(X_train, y_train), (X_valid, y_valid)], eval_metric='rmse',
            verbose=500, early_stopping_rounds=10)
    
    y_pred = model.predict(X_test, num_iteration=model.best_iteration_)
    prediction += y_pred
prediction /= n_fold

In [ ]:
from catboost import CatBoostRegressor

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20, random_state=3)

In [ ]:
model = CatBoostRegressor(iterations=1000,
                             learning_rate=0.1,
                             depth=10,
                             eval_metric='RMSE',
                             random_seed = 42,
                             bagging_temperature = 0.2,
                             od_type='Iter',
                             #metric_period = 50,
                             od_wait=20)
model.fit(X_train, y_train,
              eval_set=(X_test, y_test),
              use_best_model=True,
              verbose=True)
    
pred_train = model.predict(X_train)
pred_test = model.predict(X_test)
pred_submit = model.predict(x_test)
